### Домашнее задание Transformers Training (50 баллов)

В этом домашнем задании требуется обучить несколько Transformer-based моделей в задаче машинного перевода. Для обучения можно воспользоваться текущим проектом, так и реализовать свой пайплайн обучения. Если будете использовать проект, теги **TODO** проекта отмечают, какие компоненты надо реализовать.
В ноутбуке нужно только отобразить результаты обучения и выводы. Архитектура модели(количество слоев, размерность и тд) остается на ваш выбор.

Ваш код обучения нужно выложить на ваш github, в строке ниже дать ссылку на него. В первую очередь будут оцениваться результаты в ноутбуке, код нужен для проверки адекватности результатов. 

Обучать модели до конца не нужно, только для демонстрации, что модель обучается и рабочая - снижение val_loss, рост bleu_score.

#### Сcылка на ваш github с проектом - https://github.com/DmitryInd/pytorch-machine-translation

Ноутбук с результатами выкладывать на ваш **google диск** курса. 

### Данные

`
wget https://www.manythings.org/anki/rus-eng.zip && unzip rus-eng.zip
`

Модели нужно обучить на задаче перевода с английского на русский.

In [ ]:
# !wget https://www.manythings.org/anki/rus-eng.zip && unzip rus-eng.zip

### Вспомогательные функции

In [ ]:
# Кажется, у меня проблемы с драйверами для видеокарты, из-за чего обучение может в произвольный момент падать
# Запрет асинхронных запусков ядра не решает проблему полностью, но заметно снижает вероятность её возникновения
import os
os.environ['CUDA_LAUNCH_BLOCKING']="1"

In [ ]:
import sys  
sys.path.insert(1, "./src")

In [ ]:
import re
import torch
import numpy as np
import matplotlib.pylab as plt

if torch.cuda.is_available():
    DEVICE = 'cuda'
else:
    DEVICE = 'cpu'

In [ ]:
def print_learning_parameters(data_config, model_config):
    print(f"Parameters of the model:")
    print(f"* Learning rate is {model_config['learning_rate']}.")
    if 'div_factor' in model_config:
        print(f"* OneCycleLR with warm up and linear decrease of learning rate to division factor {model_config['div_factor']} is in use.")
    else:
        print("* Adafactor with constant learning rate is in use.")
    print(f"* Batch size is {data_config['batch_size']}.")
    if data_config['pretrained_input_tokenizer_name'] is not None:
        print(f"* The tokenizer for input values uses the pretrained vocabulary from '{data_config['pretrained_input_tokenizer_name']}'.")
    if data_config['pretrained_output_tokenizer_name'] is not None:
        print(f"* The tokenizer for output values uses the pretrained vocabulary from '{data_config['pretrained_output_tokenizer_name']}'.")
    print(f"* The number of epochs is {model_config['epoch_num']}.")
    if "pretrained_model_name" in model_config:
        print(f"* Pretrained parameters from '{model_config['pretrained_model_name']}' is used.")
    if 'emb_size' in model_config:
        print(f"* The size of embeddings/hidden states is {model_config['emb_size']}.")
    if 'num_encoder_layers' in model_config and 'num_decoder_layers' in model_config:
        print(f"* The numbers of encoder and decoder layers are {model_config['num_encoder_layers']} and {model_config['num_decoder_layers']}.")
    if 'num_heads' in model_config:
        print(f"* There are {model_config['num_heads']} heads for self-attention and encoder-decoder attention")

In [ ]:
MATCH_REGEX = re.compile(r"[+-]?\b(\d+([.]\d*)?([eE][+-]?\d+)?|[.]\d+([eE][+-]?\d+)?)\b")

def extract_values(string):
    result = dict()
    params = ['val_loss', 'train_loss', 'bleu_score']
    if "train_loss" in string:
        found_vals = [re_match[0] for re_match in re.findall(MATCH_REGEX, string)]

        if len(params) == len(found_vals):
            for name, val in zip(params, found_vals):
                result[name] = float(val)
    return result

def plot_results(train_loss_list, val_loss_list, val_bleu_list, run_name):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 5))

    ax1.plot(range(len(train_loss_list)), train_loss_list, label='train loss')
    ax1.plot(range(len(val_loss_list)), val_loss_list, label='val loss')
    ax1.set_xlabel('epoch')
    ax1.set_ylabel('loss')
    ax1.legend()

    ax2.plot(range(len(val_bleu_list)), val_bleu_list, label='val bleu')
    ax2.set_xlabel('epoch')
    ax2.set_ylabel('BLEU')
    ax2.legend()

    fig.suptitle(run_name, fontsize=20)
    plt.show()

In [ ]:
def print_demo_translations(model, val_dataloader, input_tokenizer, sentence_num=10):
    input_tensor, target_tensor = val_dataloader.__next__()
    input_tensor = input_tensor[:sentence_num]
    target_tensor = target_tensor[:sentence_num]
    with torch.no_grad():
        predicted_samples, _ = model.forward(input_tensor)
    bleu_score, actual_sentences, predicted_sentences = model.eval_bleu(predicted_samples, target_tensor)
    print(f'BLEU score on the following sentences is {bleu_score}.')
    print("Original sentence | Predicted translation | True translation")
    for in_tens, pred, actual in zip(input_tensor, actual_sentences, predicted_sentences):
        print(f"{input_tokenizer(in_tens)} | {pred} | {actual}")
    print('##############################')

### Обучение Seq2seq Transformer модель(25 баллов)

Реализуйте Seq2seq Transformer. В качестве блока трансформера можно использовать https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html. В качестве токенизатора воспользуйтесь HuggingFace токенизатор для source/target языков - https://huggingface.co/docs/transformers/fast_tokenizers
В качестве максимальной длинны возьмите предложения длинной **до 15 слов**, без каких либо префиксов. 

Не забудьте остальные элементы модели:
* Мы можем использовать 1 трансформер как энкодер - декодером будет выступать линейный слой. 
* Обучите свой BPE токенизатор - https://huggingface.co/docs/transformers/fast_tokenizers
* Матрицу эмбеддингов токенов
* Матрицу позиционных эмбеддингов
* Линейный слой проекции в target словарь
* Функцию маскирования будущих состояний attention, так как модель авто-регрессионна
* Learning rate scheduler


В качестве результатов, приложите следующие данные:
1) Параметры обучения - learning rate, batch_size, epoch_num, размерность скрытого слоя, количество слоев
2) Графики обучения - train loss, val loss, bleu score
3) Примеры переводов вашей модели(10 штук) - source text, true target text, predicted target text

In [ ]:
import yaml
from models import trainer
from data.datamodule import DataManager
from txt_logger import TXTLogger
from models.seq2seq_transformer import Seq2SeqTransformer

### Конфигурация модели и алгоритма обучения

In [ ]:
transformer_data_config = yaml.load(open("configs/data_config.yaml", 'r'),   Loader=yaml.Loader)
transformer_model_config = yaml.load(open("configs/transformer_config.yaml", 'r'),   Loader=yaml.Loader)
print_learning_parameters(transformer_data_config, transformer_model_config)

### Обучение модели

In [ ]:
tran_dm = DataManager(transformer_data_config, DEVICE)
tran_train_dataloader, tran_val_dataloader = tran_dm.prepare_data()

transformer = Seq2SeqTransformer(
    device=DEVICE,
    encoder_vocab_size=len(tran_dm.source_tokenizer.index2word),
    decoder_vocab_size=len(tran_dm.target_tokenizer.index2word),
    target_tokenizer=tran_dm.target_tokenizer,
    start_symbol=tran_dm.target_tokenizer.sos_token,
    lr=transformer_model_config['learning_rate'],
    total_steps=transformer_model_config['epoch_num']*len(tran_train_dataloader),
    emb_size=transformer_model_config['emb_size'],
    num_heads=transformer_model_config['num_heads'],
    num_encoder_layers=transformer_model_config['num_encoder_layers'],
    num_decoder_layers=transformer_model_config['num_decoder_layers'],
    dropout=transformer_model_config['dropout'],
    div_factor=transformer_model_config['div_factor']
)

tran_logger = TXTLogger(transformer_model_config['path_to_log'])
tran_trainer_cls = trainer.Trainer(model=transformer, model_config=transformer_model_config, logger=tran_logger)

if transformer_model_config['try_one_batch']:
    tran_train_dataloader = [list(tran_train_dataloader)[0]]
    tran_val_dataloader = [list(tran_val_dataloader)[0]]

tran_trainer_cls.train(tran_train_dataloader, tran_val_dataloader)

In [ ]:
file_content = open(os.path.join(transformer_model_config['path_to_log'], "progress_log.txt"), 'r').read().split("\n")
tran_train_loss_list, tran_val_loss_list, tran_val_bleu_list = [], [], []
for line in file_content:
    d = extract_values(line)
    if len(d) > 0:
        tran_train_loss_list.append(d['train_loss'])
        tran_val_loss_list.append(d['val_loss'])
        tran_val_bleu_list.append(d['bleu_score'])

In [ ]:
plot_results(tran_train_loss_list, tran_val_loss_list, tran_val_bleu_list, run_name='Transformer training')

### Итоговое качество модели/примеры переводов

In [ ]:
print(f"Final BLUE score is {tran_val_bleu_list[-1]}.")
print(f"The best BLUE score is {max(tran_val_bleu_list)} on {np.argmax(tran_val_bleu_list) + 1} step.")

In [ ]:
print_demo_translations(transformer, tran_val_dataloader, tran_dm.source_tokenizer)

### Fine-tune pretrained T5 (25 баллов)

Реализуйте Seq2seq Pretrained T5. Воспользуйтесь https://huggingface.co/docs/transformers/model_doc/t5 предобученной моделью. В качестве максимальной длинны возьмите предложения длинной **до 15 слов**, без каких либо префиксов. Архитектура модели(количество слоев, размерность и тд) остается на ваш выбор.

Не забудьте важные аспекты обучения модели:
* Взять готовый t5 токенизатор
* Resize matrix embedding - скорей всего ваша матрица эмбеддингов не будет включать эмбеддинги из вашего сета. Пример обновления матрицы эмбеддингов тут тут https://github.com/runnerup96/Transformers-Tuning/blob/main/t5_encoder_decoder.py
* Learning rate scheduler/Adafactor with constant learning rate


В качестве результатов, приложите следующие данные:
1) Параметры обучения - learning rate, batch_size, epoch_num, pretrained model name
2) Графики обучения - train loss, val loss, bleu score
3) Примеры переводов вашей модели(10 штук) - source text, true target text, predicted target text

### Конфигурация модели и алгоритма обучения

In [ ]:
t5_data_config = yaml.load(open("configs/data_config.yaml", 'r'),   Loader=yaml.Loader)
t5_model_config = yaml.load(open("configs/transformer_config.yaml", 'r'),   Loader=yaml.Loader)
print_learning_parameters(t5_data_config, t5_model_config)

### Обучение модели

In [ ]:
t5_dm = DataManager(t5_data_config, DEVICE)
t5_train_dataloader, t5_val_dataloader = t5_dm.prepare_data()

t5_model = Seq2SeqTransformer(
    device=DEVICE,
    encoder_vocab_size=len(tran_dm.source_tokenizer.index2word),
    decoder_vocab_size=len(tran_dm.target_tokenizer.index2word),
    target_tokenizer=tran_dm.target_tokenizer,
    start_symbol=tran_dm.target_tokenizer.sos_token,
    lr=transformer_model_config['learning_rate'],
    total_steps=transformer_model_config['epoch_num']*len(tran_train_dataloader),
    emb_size=transformer_model_config['emb_size'],
    num_heads=transformer_model_config['num_heads'],
    num_encoder_layers=transformer_model_config['num_encoder_layers'],
    num_decoder_layers=transformer_model_config['num_decoder_layers'],
    dropout=transformer_model_config['dropout'],
    div_factor=transformer_model_config['div_factor']
)

t5_logger = TXTLogger(t5_model_config['path_to_log'])
t5_trainer_cls = trainer.Trainer(model=t5_model, model_config=t5_model_config, logger=t5_logger)

if t5_model_config['try_one_batch']:
    train_dataloader = [list(t5_train_dataloader)[0]]
    tran_val_dataloader = [list(t5_val_dataloader)[0]]

t5_trainer_cls.train(t5_train_dataloader, t5_val_dataloader)

In [ ]:
file_content = open(os.path.join(transformer_model_config['path_to_log'], "progress_log.txt"), 'r').read().split("\n")
t5_train_loss_list, t5_val_loss_list, t5_val_bleu_list = [], [], []
for line in file_content:
    d = extract_values(line)
    if len(d) > 0:
        t5_train_loss_list.append(d['train_loss'])
        t5_val_loss_list.append(d['val_loss'])
        t5_val_bleu_list.append(d['bleu_score'])

In [ ]:
plot_results(t5_train_loss_list, t5_val_loss_list, t5_val_bleu_list, run_name='Transformer training')

### Итоговое качество модели/примеры переводов

In [ ]:
print(f"Final BLUE score is {t5_val_bleu_list[-1]}.")
print(f"The best BLUE score is {max(t5_val_bleu_list)} on {np.argmax(tran_val_bleu_list) + 1} step.")

In [ ]:
print_demo_translations(t5_model, t5_val_dataloader, t5_dm.source_tokenizer)